# Imports and Methods

In [1]:
# neccessary
import os
import math
import json
import shutil
import traceback
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint

# preprocess
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


# metrics
import scipy.stats as ss
from sklearn import metrics
# from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_squared_error

# ensembles
import xgboost
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# HyperTuning Strategies
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import RandomizedSearchCV
### explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

# ploting
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [2]:
import sklearn
sklearn.__version__

'1.0.2'

# Set Hyper Parameter Parameters

## Data preprocess

In [5]:
df = pd.read_excel("../BCB/Crispr/Codes/Python/Data Cleaner/val.xlsx",engine='openpyxl')
data = df[df.columns[1:7]]
# data=(data-data.mean())/data.std()
data = (data-data.min())/(data.max()-data.min())
X = np.array(data[data.columns[1:7]])
y = np.array(data[data.columns[0]])

data

,KO_reporter_assay,DeepCRISPR_score,CRISPRater_score,SSC_Score,sgRNA_Scorer_score,sgRNA_Designer_rsII_score
0,0.000000,0.203982,0.335595,0.305248,0.306651,0.663082
1,0.000000,0.000000,0.592578,0.383434,0.550406,0.594982
2,0.000000,0.308528,0.566042,0.318458,0.257799,0.374552
3,0.000000,0.154977,0.474262,0.461978,0.538870,0.519713
4,0.000000,0.110096,0.552275,0.502321,0.123884,0.508961
...,...,...,...,...,...,...
420,0.979445,0.820591,0.726856,0.792217,0.697362,0.840502
421,0.981501,0.732982,0.542897,0.423420,0.132310,0.634409
422,0.981501,0.859182,0.735834,0.743663,0.935299,0.722222
423,0.989723,0.197521,0.407223,0.469475,0.604574,0.731183


## Define Parameters

In [6]:
rf = RandomForestRegressor()
rfp={'bootstrap': [True, False],
     'ccp_alpha': [float("{:.2f}".format(i*0.1)) for i in range(11)],
     'criterion': ['squared_error', 'absolute_error', 'poisson'],
     'max_depth': list(np.append([None],[i*100 for i in range(2,21,2)])),
     'max_features': [None, 'auto', 'sqrt', 'log2'], 
     'max_leaf_nodes': list(np.append([None], [i for i in range(100,2001,100)])),
     'max_samples': list(np.append([None], [i for i in range(100,256,10)])),
     'min_impurity_decrease': [float("{:.2f}".format(i*0.1)) for i in range(11)],
    #  'min_impurity_split': list(np.append([None], [i for i in range(10)])),
     'min_samples_leaf': [i for i in range(1,21)],
     'min_samples_split': [i for i in range(2,21)],
     'min_weight_fraction_leaf': [float("{:.2f}".format(i*0.1)) for i in range(11)],
     'n_estimators': [i for i in range(100,2001,100)],
     'n_jobs': [-1],
     'bootstrap': [True],
     'oob_score': [True, False],
     'random_state': [42],
     'verbose': [0],
     'warm_start': [True, False],
    }

In [24]:
0.1000000000000000000001*3

0.30000000000000004

In [7]:
gb = GradientBoostingRegressor()
gbp = {'alpha': [float("{:.2f}".format(i*0.1)) for i in range(9,0,-1)],
       'ccp_alpha': [float("{:.2f}".format(i*0.1)) for i in range(11)],
       'criterion': ['friedman_mse', 'squared_error'],
       'init': [None],
       'learning_rate': [float("{:.2f}".format(i*0.1)) for i in range(1,11)],
       'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
       'max_depth': list(np.append([None],[i*100 for i in range(2,21,2)])),
       'max_features': [None, 'auto', 'sqrt', 'log2'],
       'max_leaf_nodes': list(np.append([None], [i for i in range(10,len(X),len(X)//10)])),
       'min_impurity_decrease': [float("{:.2f}".format(i*0.1)) for i in range(11)],
       'min_samples_leaf': [i for i in range(1,21)],
       'min_samples_split': [i for i in range(2,21)],
       'min_weight_fraction_leaf': [float("{:.2f}".format(i*0.1)) for i in range(6)],
       'n_estimators': [i for i in range(100,2001,100)],
       'n_iter_no_change': [None, 1, 3, 5, 7],
       'random_state': [42],
       'subsample': [float("{:.2f}".format(i*0.1)) for i in range(1,11)],
       'tol':[0.0001, 0.001, 0.01, 0.1],
       'validation_fraction': [float("{:.2f}".format(i*0.1)) for i in range(1,6)],
       'verbose': [0],
       'warm_start': [True, False]
       }

In [8]:
lr = LinearRegression()
lrp = {'copy_X': [True],
       'fit_intercept': [False, True],
       'n_jobs': [-1],
       'positive': [True, False],
       }

In [9]:
et = ExtraTreeRegressor()
etp = {'ccp_alpha': [float("{:.2f}".format(i*0.1)) for i in range(11)],
       'criterion': ['friedman_mse', 'squared_error'],
       'max_depth': list(np.append([None],[i*100 for i in range(2,21,2)])),
       'max_features': [None ,'auto', 'sqrt', 'log2'],
       'max_leaf_nodes':  list(np.append([None], [i for i in range(10,len(X),len(X)//10)])),
       'min_impurity_decrease': [float("{:.2f}".format(i*0.1)) for i in range(11)],
       'min_samples_leaf': [i for i in range(1,21)],
       'min_samples_split': [i for i in range(2,21)],
       'min_weight_fraction_leaf':  [float("{:.2f}".format(i*0.1)) for i in range(11)],
       'random_state': [42],
       'splitter':{'random', 'best'}}

In [10]:
xgbr = xgboost.XGBRegressor()
xgbrp = {'objective': 'reg:squarederror',
        'base_score': None,
        'booster': None,
        'colsample_bylevel': None,
        'colsample_bynode': None,
        'colsample_bytree': None,
        'enable_categorical': False,
        'gamma': None,
        'gpu_id': None,
        'importance_type': None,
        'interaction_constraints': None,
        'learning_rate': None,
        'max_delta_step': None,
        'max_depth': None,
        'min_child_weight': None,
        'missing': np.nan,
        'monotone_constraints': None,
        'n_estimators': 100,
        'n_jobs': None,
        'num_parallel_tree': None,
        'predictor': None,
        'random_state': None,
        'reg_alpha': None,
        'reg_lambda': None,
        'scale_pos_weight': None,
        'subsample': None,
        'tree_method': None,
        'validate_parameters': None,
        'verbosity': None}

In [11]:
xgbrf = xgboost.XGBRFRegressor()
xgbrfp = {'colsample_bynode': 0.8,
         'learning_rate': 1.0,
         'reg_lambda': 1e-05,
         'subsample': 0.8,
         'use_label_encoder': True,
         'objective': 'binary:logistic',
         'base_score': None,
         'booster': None,
         'colsample_bylevel': None,
         'colsample_bytree': None,
         'enable_categorical': False,
         'gamma': None,
         'gpu_id': None,
         'importance_type': None,
         'interaction_constraints': None,
         'max_delta_step': None,
         'max_depth': None,
         'min_child_weight': None,
         'missing': np.nan,
         'monotone_constraints': None,
         'n_estimators': 100,
         'n_jobs': None,
         'num_parallel_tree': None,
         'predictor': None,
         'random_state': None,
         'reg_alpha': None,
         'scale_pos_weight': None,
         'tree_method': None,
         'validate_parameters': None,
         'verbosity': None}

In [12]:
vr = VotingRegressor([])
estimators = []
vrp = {'estimators': [estimators[:i] for i in range(1,len(estimators)+1)],
       'n_jobs': [-1],
       'verbose': False,
       'weights': None}

In [15]:
gb.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

# Write Costum Scorings

In [12]:
def spearman(estimator, X, y):
    corr, _ = ss.spearmanr(y.argsort(), estimator.predict(X).argsort(),axis=1)
    return corr

def pearson(estimator, X, y):
    corr, _ = ss.pearsonr(y, estimator.predict(X))
    return corr

# def spearman(estimator, X, y):
#     corr = np.corrcoef(y.argsort(), estimator.predict(X).argsort())
#     return corr[0][1] if corr[0][1]>=0 else 0

# def pearson(estimator, X, y):
#     corr = np.corrcoef(y, estimator.predict(X))
#     return corr[0][1]

# Finding Best Parameters

In [13]:
SCORINGS = {#'spearman': spearman,
           #'pearson' : pearson,
           'explained_variance' : 'explained_variance',
           'max_error':"max_error",
           'neg_mean_absolute_error':"neg_mean_absolute_error",
           'neg_mean_squared_error':"neg_mean_squared_error",
           'neg_root_mean_squared_error':'neg_root_mean_squared_error',
           'neg_mean_squared_log_error':'neg_mean_squared_log_error',
           'neg_median_absolute_error':'neg_median_absolute_error',
           'r2':'r2',
           'neg_mean_poisson_deviance':'neg_mean_poisson_deviance',
           'neg_mean_gamma_deviance':'neg_mean_gamma_deviance',
           'neg_mean_absolute_percentage_error':'neg_mean_absolute_percentage_error',
           }

def clear_dir(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def save_json(jfile, path):
    with open(path, 'w') as fp:
        json.dump(jfile, fp)

def save_csv(pdfile, path):
    df = pd.DataFrame(pdfile)
    df.to_csv(path)
    
def make_folder(path):
    try:
        os.makedirs(path)
    except OSError as e:
        clear_dir(path)
#         traceback.print_exc()
            
def after_func(model, X_train, y_train, X_test, y_test, estimator_name, score):
    y_pred = model.best_estimator_.predict(X_test)
        
    print(estimator_name+':'+score, ss.spearmanr(y_test, y_pred, axis=1).correlation)
    print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0], axis=1).correlation)

def RandomizedSearchCV_hypertuning_by_score(X, y, estimator, estimator_params, scorings=SCORINGS, test_size=0.1, random_state=42, n_iter = 100, cv = 10, verbose=1, n_jobs = -1, after_estimation=None):
    estimator_name = str(type(estimator)).split('.')[-1].strip("'>")
    make_folder(estimator_name)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=random_state)
    
    scoring_names = list(scorings.keys())
    
    for score in tqdm(scoring_names):
    
        model = RandomizedSearchCV(estimator = estimator, param_distributions = estimator_params,
                                   n_iter = n_iter, cv = cv, verbose=verbose , random_state=random_state,
                                   n_jobs = -1, scoring=scorings[score], refit=scorings[score])
        
        model.fit(X_train, y_train)
        save_json(jfile=model.best_params_, path=estimator_name+'//'+score+'.json')
        save_csv(pdfile=model.cv_results_, path=estimator_name+'//'+score+'.csv' )
        
        if after_estimation != None: after_estimation(model, X_train, y_train, X_test, y_test, estimator_name, score)
        
       
    

def hypertuning():
    pass

In [ ]:
RandomizedSearchCV_hypertuning_by_score(X=X, y=y, estimator=gbas, estimator_params=gbp,
                                        scorings=SCORINGS, test_size=0.1, random_state=None,
                                        n_iter = 100, cv = 10, verbose=0, n_jobs = -1, after_estimation=after_func)

In [17]:
scoring = {#'spearman': spearman,
           #'pearson' : pearson,
           'explained_variance' : 'explained_variance',
           'max_error':"max_error",
           'neg_mean_absolute_error':"neg_mean_absolute_error",
           'neg_mean_squared_error':"neg_mean_squared_error",
           'neg_root_mean_squared_error':'neg_root_mean_squared_error',
           'neg_mean_squared_log_error':'neg_mean_squared_log_error',
           'neg_median_absolute_error':'neg_median_absolute_error',
           'r2':'r2',
           'neg_mean_poisson_deviance':'neg_mean_poisson_deviance',
           #'neg_mean_gamma_deviance':'neg_mean_gamma_deviance',
           'neg_mean_absolute_percentage_error':'neg_mean_absolute_percentage_error',
           }
model = RandomizedSearchCV(estimator = rf, param_distributions = rfp, n_iter = 100,cv = 10,verbose=1 , random_state=42,
                           n_jobs = 10, scoring=list(scoring.keys())[1], #efit=list(scoring.keys())[0]
                          )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42)

# For training, fit() is used
model.fit(X_train, y_train )

y_pred = model.best_estimator_.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
print("random_forest:", spearman(model.best_estimator_,X_test, y_test))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0] ,axis=1))

NameError: name 'rf' is not defined

In [66]:
list(scoring.keys())[0]

'explained_variance'

## Testing Parametes

In [ ]:
xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.5,
#             max_depth = 23, alpha = 10, n_estimators = 500)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=None)
# read in data
dtrain = xgboost.DMatrix(data=X_train, label=y_train)
dtest = xgboost.DMatrix( data=X_test, label=y_test)
# specify parameters via map
param = {'booster':'gbtree','max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
model = xgboost.train(param, dtrain, num_round)
# make prediction
print("random_forest:", ss.spearmanr(model.predict(dtest), y_test))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0]))

[18:41:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
random_forest: SpearmanrResult(correlation=0.4574563678670267, pvalue=2.059357684728073e-12)
State_of_the_Art: SpearmanrResult(correlation=0.48779877501496643, pvalue=3.927695026087398e-14)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=None)
model = LinearRegression()
model.fit(X_train, y_train )
print("random_forest:", ss.spearmanr(model.predict(X_test), y_test, axis=1))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0], axis=1))

random_forest: SpearmanrResult(correlation=0.46288605432882507, pvalue=0.0017747101545552756)
State_of_the_Art: SpearmanrResult(correlation=0.42648946734897303, pvalue=0.004346251616008084)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=None)
r1 = GradientBoostingRegressor()
r2 = RandomForestRegressor(n_estimators=10, random_state=1)
r3 = LinearRegression()
r4 = xgboost.XGBRegressor()
model = VotingRegressor([
    ('lr', r3),
#                           ('rf', r2),
                          ("gb", r1), 
#                          ('xgb',r4)
                        ])
model.fit(X_train, y_train )
print("random_forest:", ss.spearmanr(model.predict(X_test), y_test))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0] ,axis=1))

random_forest: SpearmanrResult(correlation=0.5024704562961543, pvalue=5.008984314604033e-15)
State_of_the_Art: SpearmanrResult(correlation=0.5027636831494182, pvalue=4.802138698944565e-15)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=None)
model = ExtraTreeRegressor()
model.fit(X_train, y_train )
print("random_forest:", ss.spearmanr(model.predict(X_test), y_test))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0] ,axis=1))

random_forest: SpearmanrResult(correlation=0.27591026543211333, pvalue=0.07331040078395479)
State_of_the_Art: SpearmanrResult(correlation=0.3222201266659473, pvalue=0.03509662651836398)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=None)
model = GradientBoostingRegressor()
model.fit(X_train, y_train )
print("random_forest:", ss.spearmanr(model.predict(X_test), y_test))
print("State_of_the_Art:", ss.spearmanr(y_test, X_test[:,0] ,axis=1))

random_forest: SpearmanrResult(correlation=0.5764772549725432, pvalue=5.2152387162701046e-05)
State_of_the_Art: SpearmanrResult(correlation=0.46645271207301536, pvalue=0.0016169113802534522)


In [80]:
model.best_params_

{'warm_start': True,
 'verbose': 0,
 'random_state': 42,
 'oob_score': True,
 'n_jobs': 1,
 'n_estimators': 100,
 'min_weight_fraction_leaf': 0.3,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'min_impurity_decrease': 0.2,
 'max_samples': 250,
 'max_leaf_nodes': 500,
 'max_features': 'log2',
 'max_depth': 1800,
 'criterion': 'poisson',
 'ccp_alpha': 0.8,
 'bootstrap': True}

In [6]:
rcParams['figure.figsize'] = 12, 4
target = 'Disbursed'
IDcol = 'ID'

def modelfit(alg, X_train, y_train, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(X_train, y_train)
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_val_score(alg, X_train, y_train, cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

#Choose all predictors except target & IDcols
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=None)

gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, X_train, y_train)

NameError: name 'GradientBoostingClassifier' is not defined

# Train with Parameters

# Test

In [15]:
[float("{:.2f}".format(i*0.1)) for i in range(10,5,-1)]

[1.0, 0.9, 0.8, 0.7, 0.6]